# Time Series Project - Store Sales
#### Cosimo Carlo Canova

## Objective :

### To predict sales of different products by store and household based on historical data, including promotional factors.

## Tasks :

###  - Analyze the dataset features: store_nbr, family, onpromotion, sales (target), date
### - Identify any relationships between variables, such as the impact of promotions on sales or seasonal differences between stores and products.